In [ ]:
from dotenv import load_dotenv
import os
import pandas as pd
import requests
from typing import Optional

load_dotenv()


def get_attendance(from_to: Optional[tuple[str, str]] = None) -> pd.DataFrame:
    return pd.DataFrame(
        [
            {
                "uuid": attendee["uuid"],
                "displayName": attendee["displayName"],
                **{
                    entry["date"]["isoYearMonthDay"]: entry["isMarkedAttended"]
                    for entry in attendee["entries"] or []
                    if pd.to_datetime(entry["date"]["isoYearMonthDay"]).day
                    in set(range(8, 15)).union(range(22, 29))
                },
            }
            for attendee in requests.get(
                os.getenv("ATTENDANCE_URL", "")
                + (f"/start/{from_to[0]}/end/{from_to[1]}" if from_to else ""),
                {"lang": "eng"},
                headers={
                    "Cookie": os.getenv("ATTENDANCE_COOKIE", ""),
                },
            ).json()["attendanceData"]["attendees"]
            if from_to
            or "becef79e-6486-4c26-a9c8-1a308448c33d" in attendee["unitOrgsCombined"]
        ]
    ).drop(columns=["displayName"] if from_to else [])


attendance = get_attendance().merge(
    get_attendance(("2025-01-25", "2025-04-27")), on="uuid", suffixes=("", "_duplicate")
)
attendance.drop(
    columns=[col for col in attendance.columns if col.endswith("_duplicate")],
    inplace=True,
)
attendance.drop(columns=["uuid"], inplace=True)
attendance.drop(columns=attendance.columns[~attendance.any()], inplace=True)
attendance.set_index("displayName", inplace=True)
attendance.index.name = None
attendance.sort_index(inplace=True)
attendance.sort_index(axis=1, inplace=True)
attendance.columns = attendance.columns.str.replace(r"202\d-", "", regex=True).str.replace("-", "/")
attendance

In [ ]:
meetings_attended = attendance.sum(axis=1)
meetings_attended

In [ ]:
for name in meetings_attended[meetings_attended > 1].index:
    print(name)

In [ ]:
for name in meetings_attended[meetings_attended == 0].index:
    print("*", name)

In [ ]:
(
    attendance[attendance.any(axis=1)]
    .sort_index(key=lambda x: meetings_attended[x])
    .replace({True: "✅", False: "❌"})
    .style
    # .set_properties(**{
    #     'font-size': '14pt',
    # })
    .set_table_styles([
        {'selector': 'th', 'props': [('font-size', '14pt')]}
    ])
)

In [ ]:
attendance.T[(attendance.T.mean() - attendance.T.ewm(span=5).mean()[-1:].T.squeeze()).sort_values(ascending=False).head(10).index]